In [1]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [2]:
school = pd.read_csv("../data/curated/school_df_21_preprocessed.csv")
mall = pd.read_csv("../data/raw/mall_info.csv")
train = pd.read_excel("../data/raw/train_station_20to21.xlsx")
postcode_SA2 = pd.read_csv("../data/raw/postcode_info.csv")
population = pd.read_csv("../data/curated/population_postcode_SA2.csv")
rent_income = pd.read_csv("../data/curated/rent_income21.csv")
park = pd.read_csv("../data/curated/park_coord.csv")

# School

In [3]:
school.head()

,Unnamed: 0,Education_Sector,School_Name,School_Type,Address_Line_1,Address_Town,Address_Postcode,LGA_Name,X,Y
0,0,Government,Alberton Primary School,Primary,21 Thomson Street,Alberton,3971,Wellington (S),146.666601,-38.617713
1,1,Government,Allansford and District Primary School,Primary,Frank Street,Allansford,3277,Warrnambool (C),142.590393,-38.386281
2,2,Government,Avoca Primary School,Primary,118 Barnett Street,Avoca,3467,Pyrenees (S),143.475649,-37.084502
3,3,Government,Avenel Primary School,Primary,40 Anderson Street,Avenel,3664,Strathbogie (S),145.234722,-36.901368
4,4,Government,Warrandyte Primary School,Primary,5-11 Forbes Street,Warrandyte,3113,Manningham (C),145.213980,-37.742675


In [4]:
# each point of the school contains the information of it's latitude, longitude and it's type.

sch_lat = school["Y"].to_list()
sch_lng = school["X"].to_list()
sch_type = school["School_Type"].to_list()

sch_point = []
for i in range(len(sch_lat)):
    sch_point.append((sch_lat[i], sch_lng[i], sch_type[i]))

# Mall

In [5]:
mall.head()

,Unnamed: 0,Mall,Stores,State,City,Latitude,Longitude
0,0,206 Bourke Street,16.0,Victoria,Melbourne,-37.812733,144.966947
1,1,Acland Court Shopping Centre,13.0,Victoria,St Kilda,-37.868967,144.980617
2,2,Altona Gate Shopping Centre,83.0,Victoria,Melbourne,-37.828989,144.846270
3,3,Balnarring Village Shopping Centre,29.0,Victoria,Balnarring,-38.373199,145.125010
4,4,Barkly Square,46.0,Victoria,Brunswick,-37.775695,144.962004


In [6]:
# each point of the mall contains the information of it's latitude, longitude and the number of stores.

mall_lat = mall["Latitude"].to_list()
mall_lng = mall["Longitude"].to_list()
mall_store = mall["Stores"].to_list()

mall_point = []
for i in range(len(mall_lat)):
    mall_point.append((mall_lat[i], mall_lng[i], mall_store[i]))

# Train stop

In [7]:
train.head()

,Fin_year,Stop_ID,Stop_name,Stop_lat,Stop_long,Pax_annual
0,FY20-21,19980,Melton Railway Station (Melton South),-37.703362,144.572217,326100
1,FY20-21,19981,Rockbank Railway Station (Rockbank),-37.729258,144.650630,57900
2,FY20-21,19982,Deer Park Railway Station (Deer Park),-37.777760,144.772307,240650
3,FY20-21,20020,Ardeer Railway Station (Ardeer),-37.783063,144.802193,23050
4,FY20-21,20287,Albury Railway Station (Albury (NSW)),-36.084263,146.924518,9950


In [8]:
# each point of the train stop contains the information of it's latitude, longitude and stop ID.

train_lat = train["Stop_lat"].to_list()
train_lng = train["Stop_long"].to_list()
train_id = train["Stop_ID"].to_list()

train_point = []
for i in range(len(train_lat)):
    train_point.append((train_lat[i], train_lng[i], train_id[i]))

# Population

In [9]:
population.head()

,Unnamed: 0,Postcode,Longitude,Latitude,SA2 name,S/T name,2001,2002,2003,2004,...,change 2011-2021 number,change 2011-2021 %,Area km2,population density 2021(persons/km2),average change %,2022,2023,2024,2025,2026
0,0,3002,144.982207,-37.818517,East Melbourne,Victoria,3731,3859,4243,4460,...,-276,-0.053,2.9,1711.2,1.014927,5036.0,5111.0,5187.0,5264.0,5343.0
1,1,3005,144.950858,-37.824608,Docklands,Victoria,154,926,1913,3089,...,9746,1.573,2.4,6522.9,1.422278,22674.0,32249.0,45867.0,65236.0,92784.0
2,2,3008,144.948039,-37.814719,Docklands,Victoria,154,926,1913,3089,...,9746,1.573,2.4,6522.9,1.422278,22674.0,32249.0,45867.0,65236.0,92784.0
3,3,3010,144.961351,-37.796152,Parkville,Victoria,5332,5327,5253,5201,...,895,0.137,4.0,1838.9,1.017541,7577.0,7710.0,7845.0,7983.0,8123.0
4,4,3050,144.954581,-37.800324,Parkville,Victoria,5332,5327,5253,5201,...,895,0.137,4.0,1838.9,1.017541,7577.0,7710.0,7845.0,7983.0,8123.0


In [10]:
# each point of the population (for each postcode) contains the information of \
    # it's latitude, longitude and growth rate from 2011 to 2021.

pop_lat = population["Latitude"].to_list()
pop_lng = population["Longitude"].to_list()
pop11_21 = population["2021"].to_list()

pop_point = []
for i in range(len(pop_lat)):
    pop_point.append((pop_lat[i], pop_lng[i], pop11_21[i]))

# Rent and Income

In [11]:
rent_income.head()

,Unnamed: 0,Postcode,Median Yearly rent price in 2021,Median income in 2021-2022,affordable index
0,0,3000,19240,28312.07,1.471521
1,1,3003,19240,20105.80,1.045000
2,2,3005,19240,55608.20,2.890239
3,3,3008,19240,55608.20,2.890239
4,4,3010,19240,54240.67,2.819162


In [12]:
# exchange the type for Postcode to string for further mergeing.

rent_income["Postcode"] = rent_income["Postcode"].astype(str)

In [13]:
# only take the attributes Postcode, Longitude and Latitude from the given csv.

postcode = pd.DataFrame(postcode_SA2, columns=["Postcode", "Longitude", "Latitude"])
postcode

,Postcode,Longitude,Latitude
0,3000,144.956776,-37.817403
1,3001,144.956776,-37.817403
2,3002,144.982207,-37.818517
3,3003,144.949592,-37.810871
4,3004,144.970161,-37.844246
...,...,...,...
3529,8111,144.970161,-37.844246
3530,8120,144.970161,-37.844246
3531,8205,144.970161,-37.844246
3532,8785,145.208504,-38.016114


# Park

In [15]:
park.head()

,Unnamed: 0,park_name,coordinates
0,0,Proposed Murray River Park (part),"(142.03935326741475, -34.14826116258584)"
1,1,Wargan-Mallee Bushland Reserve,"(142.00853908725347, -34.1535811193314)"
2,2,Proposed Murray River Park (part 2) COM,"(142.1440801948633, -34.15750991531228)"
3,3,Proposed Murray River Park (part),"(142.04389598428423, -34.15146953302827)"
4,4,Proposed Murray River Park (part),"(142.15441243467964, -34.154783832446405)"


In [16]:
# take the attributes park name, Longitude and Latitude from the given csv.

park_coor = park["coordinates"].to_list()
park_name = park["park_name"].to_list()

p_coor = []
for i in park_coor:
    i = i.replace("(","")
    i = i.replace(")", "")
    p_coor.append(i.split(','))

park_point = []
for j in range(len(p_coor)):
    park_point.append((float(p_coor[j][1]), float(p_coor[j][0]), park_name[j]))

# Map

In [17]:
# read the shp file for postcode map for Australia and rename the attribute

sf = gpd.read_file("../data/POA_2021_AUST_GDA94_SHP/POA_2021_AUST_GDA94.shp")
sf = sf.rename(columns={'POA_CODE21':'Postcode'})

In [18]:
sf.head()

,Postcode,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,0800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83680 -12.45378, 130.83683 -12.4..."
1,0810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89985 -12.36568, 130.89874 -12.3..."
2,0812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278789,0.002983,"POLYGON ((130.91914 -12.40788, 130.91830 -12.4..."
3,0820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43995, 130.85088 -12.4..."
4,0822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601833,12.564239,"MULTIPOLYGON (((136.56647 -12.08393, 136.56703..."


In [19]:
# only take the rows for Victoria state

postcode_SA2["Postcode"] = postcode_SA2["Postcode"].astype(str)

In [20]:
poa_vic = postcode_SA2["Postcode"].to_list()
poa_vic = set(poa_vic)

In [21]:
sf_new = pd.DataFrame()
for poa in poa_vic:
    df = sf[sf["Postcode"] == str(poa)]
    sf_new = pd.concat([sf_new, df])

In [22]:
sf_new.head()

,Postcode,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
743,3083,3083,AUS,Australia,18.5062,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.339318,0.001891,"POLYGON ((145.07506 -37.70815, 145.07504 -37.7..."
941,3330,3330,AUS,Australia,186.7436,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.854863,0.019136,"POLYGON ((143.65583 -37.98137, 143.65601 -37.9..."
798,3148,3148,AUS,Australia,2.9967,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.072237,0.000307,"POLYGON ((145.09986 -37.87382, 145.09983 -37.8..."
731,3068,3068,AUS,Australia,4.0529,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.096945,0.000415,"POLYGON ((144.99791 -37.79498, 144.99706 -37.7..."
1311,3919,3919,AUS,Australia,6.5625,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.150590,0.000676,"POLYGON ((145.19643 -38.33878, 145.20097 -38.3..."


In [23]:
sf_new['geometry'] = sf_new['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [24]:
# json file for postcode

geoJSON = sf_new[['Postcode','geometry']].drop_duplicates('Postcode').to_json()

In [25]:
# json file for affordable index on Postcode

re_in = gpd.GeoDataFrame(pd.merge(rent_income[['Postcode', 'affordable index']], sf, on='Postcode', how='inner'))
re_inGeo = re_in[['Postcode','geometry']].drop_duplicates('Postcode').to_json()
print(re_inGeo[:300])

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"Postcode": "3000"}, "geometry": {"type": "Polygon", "coordinates": [[[144.96138273000008, -37.82051446999998], [144.96062273000007, -37.820754489999956], [144.96000274000005, -37.82104448999996], [144.959030740


In [26]:
# create layers for different groups

poagroup = folium.FeatureGroup(name='Postcode_Layer', control=True)

popgroup = folium.FeatureGroup(name='Population_Layer', control=True)

schgroup = folium.FeatureGroup(name='School_Layer', control=True)

mallgroup = folium.FeatureGroup(name='Mall_Layer', control=True)

traingroup = folium.FeatureGroup(name='Train_Layer', control=True)

parkgroup = folium.FeatureGroup(name='Park_Layer', control=True)


In [ ]:
# add the data to the layers and then to the map

m = folium.Map(location=[-36, 145.3906], tiles="Stamen Terrain", zoom_start=7)

# Postcode layer
folium.GeoJson(
    geoJSON,
    style_function=lambda feature: {
        'fillColor': '#9ad7d2',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    },
    popup=folium.GeoJsonPopup(["Postcode"])
).add_to(poagroup)
m.add_child(poagroup)

# population layer
HeatMap(pop_point).add_to(popgroup)
m.add_child(popgroup)

# affordable index layer
affgroup = folium.Choropleth(
    geo_data=re_inGeo, # geoJSON 
    name='Affordable_Layer', # name of plot
    data=re_in, # data source
    columns=['Postcode','affordable index'], # the columns required
    key_on= 'properties.Postcode', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Affordable Index'
)
affgroup.add_to(m)

# school layer
# create mark cluster
marker_cluster_sch = MarkerCluster(name="School_Layer").add_to(schgroup)

# add data point to the mark cluster
for (lat, lng, types) in sch_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='university',color='black',icon_color='white',angle=45),
        popup=types
    ).add_to(marker_cluster_sch)

# add marker_cluster to map
m.add_child(marker_cluster_sch)


# mall layer
# create mark cluster
marker_cluster_mall = MarkerCluster(name="Mall_Layer").add_to(mallgroup)

# add data point to the mark cluster
for (lat, lng, stores) in mall_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='gift',color='orange',icon_color='white',prefix="fa"),
        popup=stores
    ).add_to(marker_cluster_mall)
    
m.add_child(marker_cluster_mall)
 
 
# train stop layer
# create mark cluster           
marker_cluster_train = MarkerCluster(name="Train_Layer").add_to(traingroup)

# add data point to the mark cluster
for (lat, lng, id) in train_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='train',color='green',icon_color='white',prefix="fa"),
        popup=id
    ).add_to(marker_cluster_train)
    
m.add_child(marker_cluster_train)


# park layer
# create mark cluster           
marker_cluster_park = MarkerCluster(name="Park_Layer").add_to(parkgroup)

# add data point to the mark cluster
for (lat, lng, name) in park_point:
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(icon='train',color='blue',icon_color='white',prefix="fa"),
        popup=name
    ).add_to(marker_cluster_park)
    
m.add_child(marker_cluster_park)

# add controller to the map
folium.LayerControl().add_to(m)

# save the map to html
m.save('../plots/liveable_affordable_poa.html')